In [4]:
import time
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import Audio
from collect_audio import read_from_file
import filters
import sys
sys.path.append("../helpers")
import os
if os.name == 'posix': #unfortunately, we can't do this with windows :(
    from collect_audio import read_from_arduino

ImportError: No module named collect_audio

# Collecting and Processing Audio Signals with Arduino and Python

In this lab we will be working with audio signals recorded with the Adafruit Metro Mini microcontroller and the electret microphone with built in amplifier. The lab will focus on loading signals into Python, signal visualization and playback, signal analysis.

## 1) Collecting Audio Signals

The first thing we need to do is get the signal loaded into Python. In other words, we need to take the analog signal (your voice) and represent it as a collection of samples in Python.

### a) Hardware Setup

Begin by wiring the microphone as in the image below with the following connections:
- OUT -> A0
- GND -> GND
- VCC -> 3V



<img src="../images/hardware_setup.jpg" height=300 width=300/>

Once you have the wiring completed, click this <a href="https://gist.github.com/gabeschamberg/b4b04254b03e774a4bf799267b8ffecf/archive/3d0581d3646fe4782251c03b19c5a3f91b3791a2.zip">link</a> to download an Arduino sketch to load to the microcontroller. This is a very basic sketch that repeteadly calls $\texttt{analogRead}$ and then passes the sample over an 115200 baud serial connection. Once the sketch is loaded to the microcontroller, use the serial plotter in the Arduino IDE to ensure that the wiring is correct!

### b) Importing Data to Python
Once you are able to see data streaming over serial, we are ready to import the data into Python. There are two ways to do this:

**1)** *read_from_file()* - Using the serial monitor, copy paste the samples into a text editor and save the file as a .txt file in the same folder as where this notebook is saved. Then, use the *read_from_file* function in the next cell to read the data, replacing the filename with the name of your file. You can try loading the file "recording.txt" as an example.

**2)** *read_from_arduino()* - ***Unfortunately, this method will not work with Windows computers.*** For non-windows users, you can use this method to directly read samples into Python from the microcontroller. Using the *read_from_arduino* function, specify the desired duration of the recording and then change the port name to match the one you find in the Arduino menu (Tools -> Port).

Uncomment the line for the method you will use. After loading the data, we'll print the number of samples it contains to confirm it worked!

In [ ]:
#data = read_from_file(filename="recording.txt")
#data = read_from_arduino(duration=10,port="/dev/tty.usbserial-ABCDEFG")

print str(data.shape[0]) + " samples loaded"

## 2) Audio Playback and Visualization
Now that we have the data, let's plot it to see what it looks like. You should be able to identify where on the signal you are making sound!

In [ ]:
plt.plot(data)
plt.xlabel('Samples')
plt.ylabel('Amplitude')

Now we want to listen to the audio we recorded. Recall that the samples that you have loaded into Python do not have time associated with them, so we need to tell the computer what the sample rate is so that it can play back the audio at the correct speed.

To figure out the sample rate, we note that our serial connection uses a baudrate of 115200. That means that it can communicate 115200 bits per second! But each sample uses 10 bits, so 115200 is NOT the sample rate. Let's estimate the sample rate by taking:

$$\frac{115200\frac{bits}{second}}{10\frac{bits}{sample}} = 11520 \frac{samples}{second} = 11.5 kHz$$

Hmm... CD players use a sample rate of $44.1kHz$, which is almost 4 times as many samples per second! Well, let's give it a shot anyway.

In [ ]:
Audio(data,rate=11520)

That didn't quite sound right... The reason is that these microcontrollers are not designed to be able to maintain a precise sample rate, much less a fast enough sample rate. Experiment with the rate to see if you can get a more normal sounding audio signal. Note that the rate cannot go below $3000Hz$, because at that point it will barely be able to create audible frequencies!

## 3) Signal Analysis
Lastly, let's do two types of analysis on the recorded signal.

### a) Frequency Analysis
As we learned above, the true sample rate of the signal is even less than $3000Hz$. For now, let's assume that it is $3000Hz$. Below we make a spectrogram of the data signal using that assumption.

In [ ]:
# create the spectogram plot
S, freqs, bins, im = plt.specgram(data, NFFT=1024, Fs=3000, noverlap=512)
# add labels
plt.xlabel('Time [s]')
plt.ylabel('Frequency [Hz]')
plt.xlim([0,len(data)/3000])
plt.ylim([0,1500]);

It seems like there is activity at all frequencies at first glance, but when we look at the y-axis we see this is not the case! Recall last week that the audio files we used had activity in frequencies above $20kHz$! Due to our low sample rate, it sounds as though the signal was passed through a low-pass filter because we can only capture low frequencies.

### b) Probability Analysis of Noise
Now let's collect real random noise and look at a histogram of the samples. Either create a new recording or use the file "noise.txt" that was created by a silent recording (so that the only signal was random noise).

In [ ]:
noise = read_from_file(filename="../files/noise.txt")
#noise = read_from_arduino(duration=10,port="/dev/tty.usbserial-ABCDEFG")

print str(noise.shape[0]) + " random samples loaded"

Now that the samples are loaded, let's plot and listen to verify that this is just like the noise that we would expect!

In [ ]:
plt.plot(noise)
plt.xlabel('Samples')
plt.ylabel('Amplitude')
Audio(noise, rate=3000)

Finally, let's take a histogram of the noisy samples. Note that this is real, physical, random noise and it looks exactly like the bell curve you see when you get your test grades back!

In [ ]:
plt.hist(noise,bins=100); 
plt.xlabel('Sample Value'); 
plt.title('Histogram of Noise Samples')
plt.ylabel('Number of Occurrences');